In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
#If you want 90% VaR, you should enter cl=0.9
#If you want 99% VaR, you should enter cl=0.99
#Historical method
def historicalVaR(returns, weights, cl=0.95):
    """
    Calculate Historical Value at Risk (VaR) at a given confidence level using historical simulation.

    Parameters:
    - returns: pd.DataFrame of asset returns (rows = time, columns = assets)
    - weights: np.array of portfolio weights (1D or 2D)
    - cl: confidence level percentile (e.g., 0.95 for 95% VaR)

    Returns:
    - float: VaR value (typically negative, indicating potential loss)
    """
    # Ensure weights is a column vector (2D shape: n x 1)
    #if weights.ndim == 1:
    #    weights = weights.reshape(-1, 1)

    # Portfolio returns (dot product of return matrix and weight vector)
    port_returns = (returns @ weights).squeeze()  # result is (T, 1) → squeeze to 1D
    
    # Remove any NaN values from the portfolio returns before sorting
    port_returns = port_returns[~np.isnan(port_returns)]
    
    #Sort the values from the lowest to the highest
    sorted_returns = np.sort(port_returns) 
    
    pos = (len(sorted_returns) - 1) *(1 -cl)
    lower_idx = int(np.floor(pos))
    upper_idx = int(np.ceil(pos))
    weight = pos - lower_idx
    
    var_pct = (1 - weight) * sorted_returns[lower_idx] + weight * sorted_returns[upper_idx]
    #var_pct= np.percentile(port_returns, （1- cl)*100)
    
    return var_pct

# Variance covariance method
# Portfolio Performance
def portfolioPerformance(weights, mean_Returns, cov_Matrix):
    
    port_mean = (mean_Returns@weights)
    
    port_std = np.sqrt((weights.T@cov_Matrix@weights)) 
    
    return [port_mean, port_std]

#If you want 90% VaR, you should enter alpha=10
#If you want 99% VaR, you should enter alpha=1
def var_parametric(port_mean, port_std,  distribution='normal', alpha=5):
    
    # because the distribution is symmetric
     if distribution == 'normal':
        # norm.ppf(alpha/100) returns a negative value (e.g., -1.645 for 95% VaR)
        var = norm.ppf(alpha / 100, loc=port_mean, scale=port_std)
        
        return var
    
     else:
        raise TypeError("Expected distribution type 'normal'")
        

# Monte Carlos method
def var_mc(returns_mean,returns_cov,Weight_list,nsim=20000,alpha=5):
    
    #Reshapes it into a column vector 
    #weights = np.array(Weight_list).reshape(-1, 1)
    
     # Simulate nsim scenarios of asset returns
    rand_normal_dist = np.random.multivariate_normal(returns_mean, returns_cov, nsim)

      # Simulate portfolio returns
    port_returns = (rand_normal_dist @ Weight_list).squeeze()  # shape (nsim,)

    # Drop missing values and calculate the alpha percentile
    return np.percentile(port_returns, alpha)
    
def stat_port(returns):
    returns_means = returns.mean()
    returns_std = returns.std()
    returns_cov = returns.cov()
    returns_corr = returns.corr()
    return [returns_means,returns_std,returns_cov,returns_corr]

In [3]:
# Read a specific sheet by index (e.g., 0 for the first sheet)
returns = pd.read_excel('Global Equity Price Change.xlsx', sheet_name=0)
df_weight = pd.read_excel('Global Equity Price Change.xlsx', sheet_name=1)

In [4]:
row_slice = returns.columns.tolist()[1:]
row_slice[0:5]

['NVDA', 'NEE', 'ASML', 'SCHW', 'UNH']

In [5]:
col_slice = df_weight.iloc[:, 0].tolist()
col_slice[0:5]

['NVDA', 'NEE', 'ASML', 'SCHW', 'UNH']

In [6]:
#Check both pages have the same companies
row_slice==col_slice

True

In [7]:
df_weight['Market_Value'] = df_weight['Price'] * df_weight['Quantity']

In [8]:
df_weight['Market_Value'].head()

0    7098500.0
1    5234600.0
2    4109400.0
3    6272448.0
4    2414181.0
Name: Market_Value, dtype: float64

In [9]:
df_total= df_weight['Market_Value'].sum()
df_total

77002272.0

In [10]:
df_weight['Weights'] = df_weight['Market_Value']/df_total
df_weight['Weights'].head()

0    0.092186
1    0.067980
2    0.053367
3    0.081458
4    0.031352
Name: Weights, dtype: float64

In [11]:
Weight_list= np.array(df_weight['Weights'])
Weight_list[0:5]

array([0.09218559, 0.06797981, 0.05336726, 0.08145796, 0.03135207])

In [12]:
# Set the dates become the index
returns.set_index('Date', inplace=True)

# Convert all values in the DataFrame to numeric.
# If a value cannot be converted (e.g., a string), it will be replaced with NaN.
returns = returns.apply(pd.to_numeric, errors='coerce')
returns=returns/100
returns.head()

,NVDA,NEE,ASML,SCHW,UNH,LOW,OXY,WFC,BUD,ADBE,...,PEP,LIN,SAP,GOOGL,WMG,SNOW,DIS,TEL,SYY,WM
Date,,,,,,,,,,,,,,,,,,,,,
2021-09-10,0.013573,-0.010601,0.006292,-0.012305,-0.022246,0.006502,-0.003590,-0.000676,-0.008804,-0.004141,...,-0.001734,-0.007227,-0.004187,-0.018566,0.000000,-0.004469,-0.009628,-0.002538,-0.018084,0.004226
2021-09-13,-0.014503,-0.004121,0.014403,0.006299,0.026268,0.008418,0.066853,0.032258,0.017938,-0.020928,...,0.001930,0.009857,0.001379,0.010339,0.001000,-0.000283,0.004671,0.006465,0.004049,0.004403
2021-09-14,0.004063,0.000709,0.019317,-0.030046,-0.004777,-0.009124,-0.013884,0.006337,-0.020701,-0.000217,...,-0.003916,-0.004976,-0.000826,0.001489,0.028472,0.002512,-0.013947,-0.010933,-0.014180,-0.003545
2021-09-15,0.004451,-0.000354,0.001419,0.020364,0.012072,0.007053,0.060883,0.012812,-0.001223,0.024914,...,0.005607,0.008303,-0.006200,0.013224,-0.017241,0.015129,0.011020,0.001796,0.006466,0.006793
2021-09-16,-0.004431,-0.007210,-0.002271,-0.005200,-0.001629,0.013958,-0.019727,-0.007719,0.003848,0.006096,...,-0.002948,-0.004610,0.004229,-0.005674,0.000247,-0.001728,-0.005802,-0.008345,0.006425,-0.011180


In [13]:
returns.shape[0]

934

In [14]:
VaR_list=[]

### Historical VaR method

In [15]:
hVaR=historicalVaR(returns, Weight_list, cl=0.95) 
hVaR

-0.02081857399903732

In [16]:
InitialInvestment = df_total
VaR_hist=InitialInvestment*hVaR
print(f'Value at Risk 95th CI    :   ${VaR_hist:,.2f}')

Value at Risk 95th CI    :   $-1,603,077.50


In [17]:
VaR_list.append(VaR_hist)

### Variance Covariance Method

In [18]:
port_mean,port_std=portfolioPerformance(Weight_list, stat_port(returns)[0], stat_port(returns)[2])
port_mean,port_std

(0.0005268623839305446, 0.013383821372994625)

In [19]:
VaR_pa=var_parametric(port_mean, port_std, distribution='normal', alpha=5)
VaR_pa

-0.02148756474391031

In [20]:
VaR_para = InitialInvestment*VaR_pa
print(f'Normal VaR 95th CI       :    ${VaR_para:,.2f}')

Normal VaR 95th CI       :    $-1,654,591.31


In [21]:
VaR_list.append(VaR_para)
VaR_list

[-1603077.4977259997, -1654591.305028192]

### Monte Carlos method

In [22]:
VaR_MC=var_mc(stat_port(returns)[0],stat_port(returns)[2],Weight_list,nsim=20000,alpha=5)

In [23]:
VaR_mc = InitialInvestment*VaR_MC
print(f'Value at Risk 95th CI    :     ${VaR_mc:,.2f}')

Value at Risk 95th CI    :     $-1,639,127.25


In [24]:
VaR_list.append(VaR_mc)

### Average VaR of 3 methods

In [25]:
VaR_list

[-1603077.4977259997, -1654591.305028192, -1639127.2530295798]

In [26]:
VaR_avg=sum(VaR_list)/len(VaR_list)
print(f'The average of 3 mothods Value at Risk 95th CI    :     ${VaR_avg:,.2f}')

The average of 3 mothods Value at Risk 95th CI    :     $-1,632,265.35


### Marginal VaR


 ### 📊 In Practice: Marginal VaR (Leave-One-Out Approximation)

When analytical derivatives are not available, Marginal VaR can be approximated using the "leave-one-out" method:

$$
\text{Marginal VaR}_i \approx \frac{\text{VaR}_{\text{portfolio}} - \text{VaR}_{\text{portfolio without asset } i}}{\text{Market Value}_i}
$$

This measures how much the total portfolio VaR would decrease if a single asset were removed, normalized by that asset’s market value.

✅ A **high Marginal VaR** means the asset increases portfolio risk.  
✅ A **low or negative Marginal VaR** means the asset reduces risk (diversifies the portfolio).



### Marginal VaR with each asset

In [27]:
df_weight1=df_weight.copy(deep=True)

In [28]:
eqty_list=[]
Marginal_VaR_list=[]
for eqty in df_weight1['Company']:
    
    eqty_market_val = df_weight1.loc[df_weight1['Company'] == eqty,'Market_Value'].iloc[0]
    market_val_left = df_total - eqty_market_val
    
    df_left = df_weight1.loc[df_weight1['Company']!= eqty].copy()
    df_left['New_weight'] = df_left['Market_Value']/market_val_left
    df_left_weight = np.array(df_left['New_weight'])
    df_left_returns=returns.drop(eqty,axis=1).copy(deep=True)
    stats = stat_port(df_left_returns)
    
    port_performance= portfolioPerformance(df_left_weight, stats[0], stats[2])
    VaR_para_new = var_parametric(port_performance[0], port_performance[1],  distribution='normal', alpha=5)*market_val_left
    VaR_his_new = historicalVaR(df_left_returns,df_left_weight, cl=0.95)*market_val_left
    VaR_mc_new = var_mc(stats[0],stats[2],df_left_weight,nsim=20000,alpha=5)*market_val_left
    
    VaR_avg_new = (VaR_para_new+VaR_his_new+VaR_mc_new)/3
    Marginal_VaR= (VaR_avg - VaR_avg_new)/eqty_market_val
    eqty_list.append(eqty)
    Marginal_VaR_list.append(Marginal_VaR)

In [29]:
Marginal_VaR_df = pd.DataFrame({'Company':eqty_list, 'Marginal VaR':Marginal_VaR_list})
Marginal_VaR_df=Marginal_VaR_df.sort_values(by='Marginal VaR', ascending=True)
Marginal_VaR_df

,Company,Marginal VaR
18,SNOW,-0.038986
0,NVDA,-0.037317
2,ASML,-0.032670
11,MU,-0.030130
9,ADBE,-0.025035
16,GOOGL,-0.021461
7,WFC,-0.019959
19,DIS,-0.019577
10,CLVT,-0.019222
3,SCHW,-0.018147


In [30]:
Marginal_VaR_df.iloc[0]

Company              SNOW
Marginal VaR   -0.0389861
Name: 18, dtype: object

At the 95% confidence level, each additional `$1` invested in SNOW increases the portfolio's potential one-day loss by approximately $0.039

### Marginal VaR with each sector


In [31]:
sector = df_weight1['Sector'].drop_duplicates()
sector

0     Information Technology
1                  Utilities
3                 Financials
4                Health Care
5     Consumer Discretionary
6                     Energy
8           Consumer Staples
10               Industrials
12    Communication Services
14                 Materials
Name: Sector, dtype: object

In [32]:
sector_list=[]
Marginal_VaR_list1=[]
for sec in sector:
    sector_market_val = df_weight1.loc[df_weight1['Sector'] == sec,'Market_Value'].sum()
    market_val_left1 = df_total - sector_market_val
    
    df_left1 = df_weight1.loc[df_weight1['Sector']!= sec].copy()
    df_left1['New_weight'] = df_left1['Market_Value']/market_val_left1
    df_left_weight1 = np.array(df_left1['New_weight'])
    
    company_left=list(df_weight1.loc[df_weight1['Sector']==sec,'Company'])
    df_left_returns1=returns.drop(company_left,axis=1).copy(deep=True)
    stats1 = stat_port(df_left_returns1)
    
    port_performance= portfolioPerformance(df_left_weight1, stats1[0], stats1[2])
    VaR_para_new = var_parametric(port_performance[0], port_performance[1],  distribution='normal', alpha=5)*market_val_left1
    VaR_his_new = historicalVaR(df_left_returns1,df_left_weight1, cl=0.95)*market_val_left1
    VaR_mc_new = var_mc(stats1[0],stats1[2],df_left_weight1,nsim=20000,alpha=5)*market_val_left1
    
    VaR_avg_new1 = (VaR_para_new+VaR_his_new+VaR_mc_new)/3
    Marginal_VaR1= (VaR_avg - VaR_avg_new1)/sector_market_val
    sector_list.append(sec)
    Marginal_VaR_list1.append(Marginal_VaR1)

In [33]:
Marginal_VaR_df1 = pd.DataFrame({'Sector':sector_list, 'Marginal VaR':Marginal_VaR_list1})
Marginal_VaR_df1 =Marginal_VaR_df1.sort_values(by='Marginal VaR', ascending=True)
Marginal_VaR_df1

,Sector,Marginal VaR
0,Information Technology,-0.029625
2,Financials,-0.018136
8,Communication Services,-0.016552
4,Consumer Discretionary,-0.015000
7,Industrials,-0.013326
5,Energy,-0.008582
1,Utilities,-0.007386
6,Consumer Staples,-0.007342
3,Health Care,-0.006794
9,Materials,-0.006755


In [34]:
Marginal_VaR_df1.loc[0]

Sector          Information Technology
Marginal VaR                -0.0296251
Name: 0, dtype: object

At the 95% confidence level, each additional `$1` invested in Information Technology sector increases the portfolio's potential one-day loss by approximately $0.030